In [1]:
import pandas as pd
import numpy as np

# Indexing

In [2]:
np.random.seed(0)
df1 = pd.DataFrame(np.random.randn(6,4), index=list('abcdef'), columns=list('ABCD'))
df1

,A,B,C,D
a,1.764052,0.400157,0.978738,2.240893
b,1.867558,-0.977278,0.950088,-0.151357
c,-0.103219,0.410599,0.144044,1.454274
d,0.761038,0.121675,0.443863,0.333674
e,1.494079,-0.205158,0.313068,-0.854096
f,-2.552990,0.653619,0.864436,-0.742165


## loc: label-based indexing

In [3]:
# get columns 'A' & 'C' of all rows with index labels 'a' or 'c' 
df1.loc[['a', 'c'], ['A', 'C']]

# returns Series
df1.loc['a', ['A', 'C']]

A    1.764052
C    0.978738
Name: a, dtype: float64

## iloc: integer positional indexing

In [4]:
# Get 1st, 2nd, and 4th row and 1st and 2nd column: 
df1.iloc[[0, 1, 3], [0, 1]]

# Get all rows and columns 2-9 if present (range is exclusive)
df1.iloc[:, 2:10]

# Get 2nd and 3rd row
df1.iloc[1:3]

,A,B,C,D
b,1.867558,-0.977278,0.950088,-0.151357
c,-0.103219,0.410599,0.144044,1.454274


## boolean mask indexing

In [5]:
df1[(df1.A < 0) & (df1['C'] > 0)]

# SQL: select C from df where A < 1
df1.loc[df1.A < 1, 'C']

c    0.144044
d    0.443863
f    0.864436
Name: C, dtype: float64

## query() method
Nice alternative for boolean indexing, even works with indexes

In [6]:
# default index name is 'index'
df1.query("A < 1 & 'a' < index <= 'd'")

,A,B,C,D
c,-0.103219,0.410599,0.144044,1.454274
d,0.761038,0.121675,0.443863,0.333674


# Grouping

In [7]:
df2 = pd.DataFrame({'A' : ['foo', 'foo', 'foo', 'bar', 'bar'],
                    'B' : ['one', 'one', 'two', 'two', 'two'],
                    'C' : [1, 2, 3, 4, 5],
                    'D' : [1, 1, 1, 1, 1]})

In [8]:
# Series with MultiIndex (A,B) and values=max(C) per A
s = df2.groupby(['A','B'])['C'].sum()
s.to_frame('sum_C')

sum_C
A   B         
bar two      9
foo one      3
    two      3

In [9]:
df2.groupby(['A','B']).agg({'C': sum, 'D': lambda x: len(x)})

C  D
A   B        
bar two  9  2
foo one  3  2
    two  3  1

In [10]:
# Series with index=order_id (like index before) and values=max(order_number)
# Data has the same number as rows as before, i.e. groupby('user_id') does not return one row per user_id!
df2['E'] = df2.groupby('A')['C'].transform(np.mean)
df2

,A,B,C,D,E
0,foo,one,1,1,2.0
1,foo,one,2,1,2.0
2,foo,two,3,1,2.0
3,bar,two,4,1,4.5
4,bar,two,5,1,4.5


# Merge & Joins

In [11]:
left  = pd.DataFrame({'A': ['A0', 'A1', 'A2'], 'B': ['B0', 'B1', 'B2']}, index=['K0', 'K1', 'K2']).rename_axis('l_index')
right = pd.DataFrame({'C': ['C0', 'C2', 'C3'], 'D': ['D0', 'D2', 'D3']}, index=['K0', 'K2', 'K3']).rename_axis('r_index')

## join
DataFrame.join defaults to a LEFT JOIN and is best suited for joining DataFrames on their indexes or joining a DataFrame with a Series.

In [12]:
# left join on indexes
left.join(right)

# inner join on indexes
left.join(right, how='inner')

# for left DF a column name to join on can be specified
# right DF is always joined using index
left.reset_index().join(right, on='l_index')

# join also works with a Series that has its name field set
# same as: left['C'] = right['C']
left.join(right['C'])

,A,B,C
l_index,,,
K0,A0,B0,C0
K1,A1,B1,NaN
K2,A2,B2,C2


## merge
DataFrame.merge defaults to an INNER JOIN and is best suited for joining on columns.

Note: when joining on columns the indexes are not preserved!

In [13]:
# inner join based on intersection of columns
left.merge(left)

# inner join on indexes using merge
left.merge(right, left_index=True, right_index=True)

,A,B,C,D
K0,A0,B0,C0,D0
K2,A2,B2,C2,D2


## semi-join with isin
In the official documentation, Series.isin is is listed in the Indexing section since it returns a boolean mask.

However, it can also be used for semi joins.

In [14]:
# SELECT * FROM left WHERE A IN (SELECT C FROM right)
left[left.A.isin(right.C)]

# semi-join using indexes
left[left.index.isin(right.index)]

# semi-join using indexes with .loc
left.loc[left.index.isin(right.index), 'B']

l_index
K0    B0
K2    B2
Name: B, dtype: object